In [1]:
import paho.mqtt.client as PahoMQTT
import time
from coopMQTT import coopMQTT
from dataset import SignalGenerator
import tensorflow as tf
import numpy as np
import pathlib
import os
import datetime
from datetime import timezone
import json
from collections import Counter

In [2]:
# DATASET PREPARATION -----------------------------------------
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# download dataset
data_dir = pathlib.Path('data/mini_speech_commands')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

# lista di test
test_list=[]
file=open("./kws_test_split.txt")
for line in file:
  test_list.append('.'+line[1:-1])

# lista di labels - da replicare in EX1
labels = open('labels.txt').readlines()[0].split() 
print(labels)

MFCC_OPTIONS = {
    'frame_length': 888, #640, 
    'frame_step': 444, #320, 
    'mfcc': True,
    'lower_frequency': 20, 
    'upper_frequency': 4000, 
    'num_mel_bins': 40,
    'num_coefficients': 10
}

# make test dataset
generator = SignalGenerator(labels, 16000, **MFCC_OPTIONS)     
test_ds = generator.make_dataset(test_list, False)

# COOPERATIVE CLIENT (RASP)----------------------------------------------------------------
timestamp = int(datetime.datetime.now(timezone.utc).timestamp())
device_name = 'rasp'

subscribe_topics, publish_topic = ['/group11IOTHW3/model/1', 
                                   '/group11IOTHW3/model/2',
                                   '/group11IOTHW3/model/3',
                                   '/group11IOTHW3/model/4'], '/group11IOTHW3/prep_sample'

rasp = coopMQTT('raspClient', subscribe_topics, publish_topic, 4, len(test_ds))
rasp.start()

for i, (mfcc, label) in enumerate(test_ds):
    senml_msg = {
        "bn" : device_name,
        "bt" : timestamp,
        "e" :[{"n":"sample_"+str(i), "u":"/", "t":0, "vd":mfcc.numpy().tolist()}]
        }
    senml_msg = json.dumps(senml_msg)
    rasp.myPublish(publish_topic, senml_msg, i, label.numpy())
    time.sleep(0.15) # tune this parameter

print('last send:', i)
while not rasp.last_received:
    pass
print('okokok')
rasp.stop()
accuracy = rasp.running_corrects/len(test_ds)*100
print(accuracy)

['down', 'stop', 'right', 'left', 'up', 'yes', 'no', 'go']
Connected to mqtt.eclipseprojects.io with result code: 0
0
0
0
0
1
1
1
1
2
2
3
3
4
4
2
3
4
5
6
7
8
9
10
5
6
7
8
9
10
5
6
7
8
9
10
2
3
4
11
12
11
12
13
14
5
6
7
8
9
10
13
14
11
12
13
14
15
11
12
15
13
14
15
16
16
16
17
15
16
17
18
17
17
18
18
19
20
19
18
19
19
20
21
20
20
21
22
21
22
22
23
23
21
22
23
24
24
23
24
24
25
25
25
26
26
25
27
28
27
28
26
27
26
27
28
29
28
29
30
30
29
30
29
30
31
32
31
32
33
31
32
31
32
33
34
33
33
34
34
34
35
35
35
36
36
37
36
37
35
36
37
38
38
37
38
39
39
39
38
40
40
39
40
40
41
41
41
41
42
43
42
43
42
44
43
44
42
43
44
44
45
45
45
45
46
46
46
46
47
48
47
47
47
48
48
48
49
50
49
50
49
50
49
50
51
52
51
52
51
52
51
52
53
53
53
54
54
53
54
55
54
55
56
55
55
56
56
57
58
57
58
59
57
56
58
57
59
60
59
58
60
59
60
61
60
61
61
62
63
62
63
64
64
61
62
63
64
62
63
64
65
65
65
66
66
66
65
67
68
67
67
68
69
66
69
70
68
69
70
70
71
67
68
69
71
71
72
73
70
71
72
73
72
73
74
72
73
74
75
74
75
76
74
75
75
76
76
76


533
533
534
534
534
534
535
535
536
535
535
536
537
536
537
536
537
538
537
538
538
539
538
539
539
539
540
541
540
540
541
540
541
542
541
542
542
543
543
542
543
543
544
544
545
544
546
545
545
546
544
545
546
547
546
547
547
547
548
549
548
548
548
549
550
549
549
550
550
551
551
551
552
552
550
551
552
553
554
552
553
553
554
554
555
555
553
554
555
556
555
557
558
556
556
557
557
556
557
558
558
558
559
559
559
560
560
560
561
561
559
560
561
561
562
562
562
563
563
562
563
564
563
564
565
564
564
565
565
566
566
567
567
566
567
565
566
567
568
568
568
568
569
569
569
569
570
570
571
570
570
571
572
571
572
571
572
573
572
573
573
573
574
574
574
574
575
575
575
576
575
576
576
576
577
577
577
578
578
577
578
579
579
578
579
579
580
580
580
581
581
582
580
581
582
582
581
583
582
583
583
583
584
584
585
585
586
584
584
585
586
586
587
588
589
585
586
587
588
587
587
588
588
589
590
590
591
592
589
591
589
590
591
592
590
591
592
593
594
593
594
593
594
592
593
594
595
596
595
596


In [ ]:
rasp.stop()